In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
from dftorch.DM_Fermi_x import DM_Fermi_x
import torch
import numpy as np
import sys
import os
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
from seqm.seqm_functions.read_xyz import read_xyz
import scipy.io as sio
import math
import pandas as pd
import importlib
import matplotlib.pyplot as plt

import dftorch
import importlib
import dftorch.CoulombMatrix
importlib.reload(dftorch.CoulombMatrix)
from dftorch.CoulombMatrix import CoulombMatrix_vectorized, Ewald_Real_Space_vectorized
from dftorch.SCF import SCF, SCF_adaptive_mixing, SCFx

from dftorch.H0andS import H0_and_S_vectorized, H0_and_S_vectorized_OLD_FOR_POLY
from dftorch.Constants import Constants, ConstantsTest
from dftorch.nearestneighborlist import vectorized_nearestneighborlist, vectorized_nearestneighborlist_batch
from dftorch.Energy import Energy, EnergyShadow
from dftorch.Tools import fractional_matrix_power_symm
from dftorch.Forces import Forces, ForcesShadow
from dftorch.BondIntegral import *
from dftorch.Tools import ordered_pairs_from_TYPE
from dftorch.Structure import Structure
from dftorch.DM_Fermi import DM_Fermi
from dftorch.Fermi_PRT import Canon_DM_PRT, Fermi_PRT
from dftorch.Kernel_Fermi import Kernel_Fermi
from dftorch.MD import initialize_velocities

from dftorch.io import read_xyz_traj_data, read_xyz, write_XYZ_trajectory
import time
import matplotlib.colors as mcolors

from dftorch.RepulsiveSpline import get_repulsion_energy
torch.set_default_dtype(torch.float32)
print(torch.cuda.memory_allocated() / 1e9, 'GB')

0.0 GB


In [ ]:
%%time
torch.autograd.set_detect_anomaly(True)

# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'
#const = Constants('/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1').to(device)
const = Constants('/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set').to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!
Te = 3000.0                       # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

req_grad = False


species, coordinates = read_xyz(['COORD.xyz'], sort=False) #Input coordinate file
LBox = torch.tensor([9,4,4], device=device)
TYPE = torch.tensor(species[0], dtype=torch.int64, device=device)
RX = torch.tensor(coordinates[0,:,0], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RY = torch.tensor(coordinates[0,:,1], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RZ = torch.tensor(coordinates[0,:,2], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
R_tensor, R_orb, coeffs_tensor, R_rep_tensor, rep_splines_tensor = get_skf_tensors(TYPE, const)


structure = Structure(TYPE, RX, RY, RZ, LBox, const, charge = 0, device=device)
# but first the neighborlist


nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, nrnnStruct, \
        neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type  = \
        vectorized_nearestneighborlist(structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox,
        8.0, structure.Nats, const, upper_tri_only=False, remove_self_neigh=False, min_image_only=False, verbose=True);



species, coordinates = read_xyz(['COORD.xyz', 'COORD.xyz'], sort=False) #Input coordinate file
LBox = torch.tensor([9,4,4], device=device)
TYPE = torch.tensor(species, dtype=torch.int64, device=device)
RX = torch.tensor(coordinates[:,:,0], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RY = torch.tensor(coordinates[:,:,1], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RZ = torch.tensor(coordinates[:,:,2], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)

nrnnlist_b, nndist_b, nnRx_b, nnRy_b, nnRz_b, nnType_b, nnStruct_b, nrnnStruct_b, \
        neighbor_I_b, neighbor_J_b, IJ_pair_type_b, JI_pair_type_b  = \
        vectorized_nearestneighborlist_batch(TYPE, RX, RY, RZ, structure.LBox,
        8.0, structure.Nats, const, upper_tri_only=False, remove_self_neigh=False, min_image_only=False, verbose=True);

/home/maxim/anaconda3/envs/hipnn_2/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
<timed exec>:7: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3725.)


/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-C.skf
  t <neighbor list> 0.1 s

  t <batched neighbor list> 0.019 s  (B=2, N=20)
CPU times: user 2.75 s, sys: 385 ms, total: 3.13 s
Wall time: 961 ms


In [5]:
neighbor_I.shape

torch.Size([2, 20, 261])

In [4]:
neighbor_I

tensor([[[ 0,  0,  0,  ...,  0, -1, -1],
         [ 1,  1,  1,  ..., -1, -1, -1],
         [ 2,  2,  2,  ..., -1, -1, -1],
         ...,
         [17, 17, 17,  ..., 17, -1, -1],
         [18, 18, 18,  ..., -1, -1, -1],
         [19, 19, 19,  ..., -1, -1, -1]],

        [[ 0,  0,  0,  ...,  0, -1, -1],
         [ 1,  1,  1,  ..., -1, -1, -1],
         [ 2,  2,  2,  ..., -1, -1, -1],
         ...,
         [17, 17, 17,  ..., 17, -1, -1],
         [18, 18, 18,  ..., -1, -1, -1],
         [19, 19, 19,  ..., -1, -1, -1]]], device='cuda:0')

In [4]:
batch_for_pairs

tensor([0, 0, 0,  ..., 1, 1, 1], device='cuda:0')

In [ ]:
%%time
torch.autograd.set_detect_anomaly(True)

# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'
#const = Constants('/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1').to(device)
const = Constants('/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set').to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!
Te = 3000.0                       # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

# species, coordinates = read_xyz(['3HDP_Ni.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([60,61,62], device=device)

# species, coordinates = read_xyz(['protein.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([100,100,100], device=device)

species, coordinates = read_xyz(['COORD.xyz'], sort=False) #Input coordinate file
LBox = torch.tensor([9,4,4], device=device)

# species, coordinates = read_xyz(['COORD_8WATER.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([20,20,20], device=device)

# species, coordinates = read_xyz(['COORD_H.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([90,40,40], device=device)

# species, coordinates = read_xyz(['C.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([90,40,40], device=device)

# species, coordinates = read_xyz(['COORD_Ni.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([9,4,4], device=device)

# species, coordinates = read_xyz(['C2.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([80,40,40], device=device)

# species, coordinates = read_xyz(['C840.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([50,50,50], device=device) #36

# species, coordinates = read_xyz(['solvated_1600.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([33,33,33], device=device)
#LBox = torch.tensor([100,100,100], device=device)

# species, coordinates = read_xyz(['gs_solvated_4k.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([100,100,100], device=device)

# species, coordinates = read_xyz(['solvated_ac_2k.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([27.2,27.2,27.2], device=device)

# species, coordinates = read_xyz(['solvated_ac_2k_relaxed.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([27.2,27.2,27.2], device=device)

# species, coordinates = read_xyz(['solvated_ac_2k_react.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([27.2,27.2,27.2], device=device)

# species, coordinates = read_xyz(['solvated_ac_glyc_1k_react.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([20,20,20], device=device)

# species, coordinates = read_xyz(['NaCl.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([27,27,27], device=device)

# species, coordinates = read_xyz(['NaCl_1.xyz'], sort=False) #Input coordinate file
# LBox = torch.tensor([27,27,27], device=device)



req_grad = False
TYPE = torch.tensor(species[0], dtype=torch.int64, device=device)
RX = torch.tensor(coordinates[0,:,0], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RY = torch.tensor(coordinates[0,:,1], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
RZ = torch.tensor(coordinates[0,:,2], device=device, dtype=torch.get_default_dtype(),
                    requires_grad=req_grad)
R_tensor, R_orb, coeffs_tensor, R_rep_tensor, rep_splines_tensor = get_skf_tensors(TYPE, const)

#const.Es_dict_read = const.Es_dict_read - 0.01
const.set_params(r_grad=True)


structure = Structure(TYPE, RX, RY, RZ, LBox, const, charge = 0, device=device)
# but first the neighborlist


nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, nrnnStruct, \
        neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type  = \
        vectorized_nearestneighborlist(structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox,
        8.0, structure.Nats, const, upper_tri_only=False, remove_self_neigh=False, min_image_only=False, verbose=True);
        
# Get Hamiltonian, Overlap, atomic DM = D0 (vector only), etc, 
D0, H0, dH0, S, dS = H0_and_S_vectorized(
                                structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox, structure.Nats,
                                structure.diagonal, structure.H_INDEX_START,structure.H_INDEX_END, structure.Znuc,
                                nrnnlist, nnRx, nnRy, nnRz, nnType,
                                const, neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type,
                                R_orb, coeffs_tensor,
                                verbose=True)
del nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, nrnnStruct, neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type

# D0, H0, dH0, S, dS = H0_and_S_vectorized_OLD_FOR_POLY(structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox, structure.Nats,
#                                 structure.diagonal, structure.H_INDEX_START,structure.H_INDEX_END, structure.Znuc,
#                                 nrnnlist, nnRx, nnRy, nnRz, nnType,
#                                 const, neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type,
#                                 R_orb, coeffs_tensor,
#                                 verbose=True)

Vr, dVr = get_repulsion_energy(
                        R_rep_tensor, rep_splines_tensor,
                        structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox, 6.0, structure.Nats, 
                        const,verbose=True);

/home/maxim/anaconda3/envs/hipnn_2/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
<timed exec>:7: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3725.)


/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-C.skf
  t <neighbor list> 0.1 s

H0_and_S
  Do H off-diag
  t <dR and pair mask> 0.0 s

  t <SKF> 0.0 s

  Do H and S
  t <H and S> 0.1 s

  t <D0> 0.0 s

H0_and_S t 0.1 s

  t <neighbor list> 0.0 s

CPU times: user 2.81 s, sys: 424 ms, total: 3.23 s
Wall time: 1.06 s


In [3]:
%%time
# Parameters for the Coulomb summations
Coulomb_acc = 1e-4
TIMERATIO = 10

COULVOL = LBox[0] * LBox[1] * LBox[2]
SQRTX = math.sqrt(-math.log(Coulomb_acc))
CALPHA = math.sqrt(math.pi) * ((TIMERATIO * structure.Nats / (COULVOL ** 2)) ** (1.0 / 6.0))
COULCUT = SQRTX / CALPHA

#### or ####
COULCUT = 12.0
CALPHA = SQRTX/COULCUT
############

if COULCUT > 50.0:
    COULCUT = 50.0
    CALPHA = SQRTX / COULCUT

#Rcut = 10.42
Rcut = COULCUT

# Get full Coulomb matrix. In principle we do not need an explicit representation of the Coulomb matrix C!
nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, \
nrnnStruct, neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type = \
vectorized_nearestneighborlist( structure.TYPE, structure.RX, structure.RY, structure.RZ, structure.LBox, Rcut, structure.Nats, const,
                                upper_tri_only=False, remove_self_neigh=False, verbose=True)
                                
C, dCC = CoulombMatrix_vectorized(structure, structure.RX,structure.RY,structure.RZ,structure.LBox,
                                    structure.Hubbard_U, structure.Hubbard_U_sr, structure.TYPE,structure.Nats,
                                    structure.HDIM,Coulomb_acc,TIMERATIO,nnRx,nnRy,nnRz,nrnnlist,nnType, neighbor_I, neighbor_J,
                                    structure.H_INDEX_START,structure.H_INDEX_END,
									CALPHA, COULCUT, verbose=True)
del nrnnlist, nndist, nnRx, nnRy, nnRz, nnType, nnStruct, nrnnStruct, neighbor_I, neighbor_J, IJ_pair_type, JI_pair_type
COULCUT

  t <neighbor list> 0.0 s

CoulombMatrix_vectorized
  Do Coulomb Real
  Coulomb_Real t 0.0 s
  Doing Coulomb k
   LMAX: 2
   0
   1


   2
  Coulomb_k t 0.1 s

CPU times: user 64.4 ms, sys: 39.3 ms, total: 104 ms
Wall time: 99.7 ms


12.0

In [4]:
#with torch.no_grad():
if 1:
	H, Hcoul, Hdipole, D, q, f = SCF(
		structure, D0,
		H0, S, Efield, C,
		structure.RX, structure.RY, structure.RZ, structure.Nocc,
		structure.Hubbard_U, structure.Znuc, Te, alpha=0.3, acc=1e-5, MAX_ITER=200, debug=False
	)



Etot,Eband0,Ecoul,Edipole,S_ent = Energy(H0, structure.Hubbard_U, Efield, D0, C, D, q,
											structure.RX, structure.RY, structure.RZ, f, Te) # Energy calculation - 2*Te*S_ent
Etot+Vr ,Eband0,Ecoul,Edipole,S_ent, Vr

### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, device='cuda:0')
  update q 0.0 s
Res = 0.340007663, dEb = 415.442687988, dEc = 0.297992468, t = 0.0 s

Iter 2
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, device='cuda:0')
  update q 0.0 s
Res = 0.191828161, dEb = 0.040191650, dEc = 0.045821428, t = 0.0 s

Iter 3
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, device='cuda:0')
  update q 0.0 s
Res = 0.108429655, dEb = 0.020141602, dEc = 0.024240136, t = 0.0 s

Iter 4
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, device='cuda:0')
  update q 0.0 s
Res = 0.061422605, dEb = 0.010284424, dEc = 0.013245463, t = 0.0 s

Iter 5
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, device='cuda:0')
  update q 0.0 s
Res = 0.034868810, dEb = 0.005950928, dEc = 0.007382512, t = 0.0 s

Iter 6
  Hcoul 0.0 s
  DM_Fermi 0.0 s
  Z@Dorth@Z.T 0.0 s
tensor(0.0002, 

(tensor(-328.6055, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(-338.9343, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.1975, device='cuda:0'),
 tensor(-0., device='cuda:0'),
 tensor(5.1822e-10, device='cuda:0'),
 tensor(10.1314, device='cuda:0'))

In [5]:
(Etot).backward()

In [6]:
force = const.Es.grad
force[1].item()


10.243340492248535

In [ ]:
force = const.Es.grad
force[1].item()


10.243386268615723

In [ ]:
force = const.Es.grad
force[1].item()


10.2433443069458